In [47]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import os
import shutil
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.models import load_model

In [22]:
path  = 'G:/Sorghum/'


In [34]:
train_df = pd.read_csv(path +'train_cultivar_mapping.csv')
train_df = trpath  = 'G:/Sorghum/'
ain_df.dropna()

In [35]:
IMG_SIZE = 256
BATCH_SIZE = 8

In [36]:
train_df['cultivar']=train_df['cultivar'].astype(str)

In [37]:
train_df

,image,cultivar
0,2017-06-16__12-24-20-930.png,PI_257599
1,2017-06-02__16-48-57-866.png,PI_154987
2,2017-06-12__13-18-07-707.png,PI_92270
3,2017-06-22__13-18-06-841.png,PI_152651
4,2017-06-26__12-56-48-642.png,PI_176766
...,...,...
22189,2017-06-16__12-27-16-266.png,PI_170787
22190,2017-06-28__11-19-57-454.png,PI_156393
22191,2017-06-28__10-20-32-417.png,PI_152923
22192,2017-06-28__12-47-02-714.png,PI_257599


In [39]:
# Set paths and split ratio
data_path = 'G:/Sorghum/train_images'
csv_path = 'G:/Sorghum/train_cultivar_mapping.csv'
split_ratio = 0.2

# Load csv file
df = pd.read_csv(csv_path)

# Split data into training and validation sets
train_df, val_df = train_test_split(df, test_size=split_ratio, random_state=42)


In [40]:
train_df['cultivar']=train_df['cultivar'].astype(str)
val_df['cultivar']=val_df['cultivar'].astype(str)

In [41]:
train_df.count()data_path = 'G:/Sorghum/train_images'
csv_path = 'G:/Sorghum/train_cultivar_mapping.csv'
split_ratio = 0.2

# Load csv file
df = pd.read_csv(csv_path)

# Split data into training and validation sets
train_df, val_df = train_test_split(df, test_size=split_ratio, random_state=42)


image       17755
cultivar    17755
dtype: int64

In [42]:
val_df.count()

image       4439
cultivar    4439
dtype: int64

In [43]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=path + 'train_images',
    x_col='image',
    y_col='cultivar',
    class_mode='sparse',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

valid_generator = train_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=path + 'train_images',
    x_col='image',
    y_col='cultivar',
    class_mode='sparse',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

C:\Users\ali\anaconda3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


Found 17754 validated image filenames belonging to 100 classes.
Found 4439 validated image filenames belonging to 100 classes.


In [44]:
base_model = keras.applications.EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [45]:
base_model.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 256, 256, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 256, 256, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 257, 257, 3)  0           normalization_1[0][0]            
_____________________________________________________________________________________

In [46]:


inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
output = base_model(inputs)
output = GlobalAveragePooling2D()(output)
output = Dense(100, activation='softmax')(output)

model = keras.Model(inputs, output)

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
efficientnetb4 (Functional)  (None, 8, 8, 1792)        17673823  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1792)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               179300    
Total params: 17,853,123
Trainable params: 17,727,916
Non-trainable params: 125,207
_________________________________________________________________


In [22]:
history = model.fit(train_generator, validation_data=valid_generator, epochs=10)

Epoch 1/10
2220/2220 [==============================] - 1160s 516ms/step - loss: 3.2400 - accuracy: 0.1956 - val_loss: 2.4567 - val_accuracy: 0.3557
Epoch 2/10
2220/2220 [==============================] - 1068s 481ms/step - loss: 1.8976 - accuracy: 0.4585 - val_loss: 2.2954 - val_accuracy: 0.4093
Epoch 3/10
2220/2220 [==============================] - 1105s 498ms/step - loss: 1.4163 - accuracy: 0.5820 - val_loss: 1.2478 - val_accuracy: 0.6265
Epoch 4/10
2220/2220 [==============================] - 1142s 514ms/step - loss: 1.1577 - accuracy: 0.6543 - val_loss: 1.1019 - val_accuracy: 0.6578
Epoch 5/10
2220/2220 [==============================] - 1296s 584ms/step - loss: 0.9662 - accuracy: 0.7072 - val_loss: 1.1426 - val_accuracy: 0.6691
Epoch 6/10
2220/2220 [==============================] - 1038s 468ms/step - loss: 0.8522 - accuracy: 0.7397 - val_loss: 0.8030 - val_accuracy: 0.7569
Epoch 7/10
2220/2220 [==============================] - 1063s 479ms/step - loss: 0.7223 - accuracy: 0.7768

In [24]:
model.fit(train_generator, validation_data=valid_generator, epochs=5)

Epoch 1/5
2220/2220 [==============================] - 1112s 501ms/step - loss: 0.4973 - accuracy: 0.8462 - val_loss: 0.6065 - val_accuracy: 0.8234
Epoch 2/5
2220/2220 [==============================] - 1280s 576ms/step - loss: 0.4639 - accuracy: 0.8536 - val_loss: 0.3869 - val_accuracy: 0.8831
Epoch 3/5
2220/2220 [==============================] - 1125s 507ms/step - loss: 0.4441 - accuracy: 0.8602 - val_loss: 0.3825 - val_accuracy: 0.8799
Epoch 4/5
2220/2220 [==============================] - 1088s 490ms/step - loss: 0.4118 - accuracy: 0.8711 - val_loss: 0.4459 - val_accuracy: 0.8653
Epoch 5/5
2220/2220 [==============================] - 1161s 523ms/step - loss: 0.3799 - accuracy: 0.8783 - val_loss: 0.3960 - val_accuracy: 0.8822


In [25]:
model.save("g:/effiecient.h5")

In [48]:
loaded_model = load_model("g:/effiecient.h5")